# Data Carpentry with `Python`, part 2

Recall our messy data from part one and all the manual steps we needed to clean things up.

![title](../images/messy2.png)

### The Data

This is only a partial dataset on several [species of desert rodents](http://esapubs.org/archive/ecol/E090/118/Portal_rodent_metadata.htm). 
The files that we have used before have all been `csv`s, but `Python` can handle a lot more types of file formats. 
The file we will be working with today is in `.xls` format, otherwise known as an Excel file. 

To clean up this data, we are going to use Pandas and the `xlrd` (Excel Reader) library.
  * External API Link: https://pypi.python.org/pypi/xlrd
  
  

In [ ]:
# import the required packages

# xlrd is a package for developers to extract data from
# Excel spreadsheets. https://pypi.python.org/pypi/xlrd

import pandas as pd 
import xlrd 

#### Read the File with Pandas

One thing to note here, we have yet to see the `header` parameter used. 
Normally, when a data frame object is created, the first row in the file is interpreted to be the header row, 
and often this is okay. 
But in its current state, our data is in no place to have a header row yet. 
Plus, the first row in the dataset for this Excel file is an empty line.
When there aren't any data to be the header row, 
`pandas` defaults the names to a series of **`Unnamed:`**s or numbers.

In [ ]:
file = pd.read_excel('/dsa/data/all_datasets/messy_survey.xls', header = None)

# now it is saved to the object `file`

file

#### Remember, we are essentially dealing with three different tables in the exact same file. 

You will notice all of the `NaN` values. 
This stands for "Not a Number" and is the default value for those cells that don't contain data.

Recall our steps for file clean up:
  1. Remove _no data_ columns
  1. Add species column, and fill down
  1. Remove non-header rows
  1. Remove rows between sub-tables to create the unified table.




# Step 1:  Remove 'no data' columns

These are the first two columns of our data.
We can do this by selecting out only the columns with data.
Column and row indexes start at 0; and with a little `pandas` knowledge we can select out a range of columns.

This is done with the `.iloc[ rows , columns ]` syntax.

**Example**
```
file.iloc[ 0: , 2: ]
```
In this case, we are asking for all rows because it is "`0:`", and columns numbered 2 to the end with "`2:`"


**Note:** the `.head()` function below just shows the top few rows in the Pandas data frame.


In [ ]:
file.iloc[ 0:, 2: ].head(10)

**Example**
```
file.iloc[ 5: , 0: ]
```
In this case, we are asking for all columns because it is "`0:`", and rows numbered 5 to the end with "`5:`"


In [ ]:
file.iloc[ 5: , 0: ].head()

#### Resetting Indexes after selection
The `.reset_index(drop = True)` function on the Pandas data frame will reset the indexes on the rows.
Notice the output above, the first row listed is indexed as **5**.

And below, we can _fix_ it to be back to 0-based indexing for the rows.

In [ ]:
file.iloc[ 5: , 0: ].reset_index(drop = True).head()

Wait weren't we trying to complete step 1?
**Notice that most of Step 3 was just accomplished!**

## Combining step 1 & partial-step 3 of our process!

To keep our source `file` data as-is, we will cut the columns and create a new variable with the result.
We are going to call this modified data frame **`messy`**.


In [ ]:
# Create new variable
messy = file.iloc[5:,2:].reset_index(drop = True)
messy.head(20)  # option number of rows to show in head

# Step 2: Add species column, and fill down

So this next section may be a little complicated, particularly if you are new to programming, 
but we will take this step-by-step.

First we will define the species cell values, which we saw are:
  * `Species: DM`
  * `Species: DO`
  * `Species: DS`

We will put these into a list as the names of the sub tables.

In [ ]:
table_names = ["Species: DM", "Species: DO", "Species: DS"]
table_names

As you learn programming, you learn tricks and patterns to solve certain types of problems.
The need to fill a value down to a point then reset to a new value is a common pattern of tasks.

Notice above that our column index 2 (heading does not reset like rows) holds the sub-table names we put into our list. Lets investigate that column.

In [ ]:
messy[2]

We can use the `.isin()` function of the Pandas data frame to test if each value is in a list of values.
Recall our list of values:
```
['Species: DM', 'Species: DO', 'Species: DS']
```

This gives us a list of True or False values going down the column based on the existence of the value in the list.

In [ ]:
messy[2].isin(table_names)

#### So... 

What can we do with this list of values?
Well, as it happens, there are a couple of rules of computers that work to our advantage.
  * `True` = 1
  * `False` = 0
  
This allows us to process this `True/False` column with another function, _cumulative summation_, `.cumsum()`.

So, adding True and False is `1 + 0` and if we keep accumulating down the column, every time we hit `True`, the value increases by 1.

A couple of examples.

In [ ]:
pd.DataFrame([True, False, True]).cumsum()

In [ ]:
pd.DataFrame([True, False, True, False, True, True, True, False, False]).cumsum()

#### Therefore... 

We can use this technique to apply labels, where group 1 is all the `Species: DM`, 2 is `Species: DO`, and 3 is `Species: DS`. This can be done because of the shape of our data. If you recall `Species: DM` is a "table" with the excel file then `Species: DO` comes next and finally `Species: DS`

In [ ]:
# Capture this processing (label creation) as a new variable, groups
groups = messy[2].isin(table_names).cumsum()
print(groups)

We now have list of values we can use to label the rows.
We can use the Pandas data frame `.groupby()` function to partition a data frame into sets of rows associated with a column value.

In this case, the rows that allign to the number 1 will be one group, 2 another, and 3 a third group of rows.

The result is going to be a new type of GroupBy data frame:
```
pandas.core.groupby.DataFrameGroupBy
```


In [ ]:
messy.groupby(groups)

This code will expand back out the data frame Group By object to show you the effects.

In [ ]:
# for each element in set
# k     g
# 1 -> { row 0 , row 12}
# 2 -> { row 13 , row 21}
# 3 -> {}
for k,g in messy.groupby(groups):
    print("k = {}, g = {}".format(k,g))

#### Notice that ... 
We have now set up the data into 3 sub-tables, K = 1, 2, and 3.

So, let's use some more appropriate variable names:
  * `species`  as the index from the cumsum() function.
  * `rows` as the collection of rows that have same label.


In [ ]:
for species,rows in messy.groupby(groups):
    print("species = {}, rows = {}".format(species,rows))

#### Looking at the rows... 
The first row and column of each sub-table has the Species name.
We can access the cell with the Pandas data frame `.iloc[row,col]`

In [ ]:
for species,rows in messy.groupby(groups):
    print(rows.iloc[0,0])

Similarly, we can look at all the rows from 1 to the end with the `.iloc[rows]`, single argument function.

#### NOTE: This completes step 3 by removing the Species name row.  

In [ ]:
for species,rows in messy.groupby(groups):
    print(rows.iloc[1:])

Based on these two manipulations, we can create a `Key->Value` data structure, called a **dictionary**

In [ ]:
tables = {} # an empty dictionary variable named tables

for species,rows in messy.groupby(groups):  # For each species and its rows:
    tables[rows.iloc[0,0]] = rows.iloc[1:]  # store into the tables variable
   
print(tables)

#### In Python, there is always an alternative way to accomplish a task:

In [ ]:
# This creates the same dictionary as above, using a "dictionary comprehension" (aka fancy Python)
tables = {rows.iloc[0,0]: rows.iloc[1:] for species,rows in messy.groupby(groups)}

############
# t={v.iloc[0,0]: v.iloc[1:] for k,v in m.groupby(g)}
############


print(tables)

#### What do these tables look like now?

Let's take a pause in our cleaning process, and print out what these separated tables look like now. We will create a for loop that iterates over each table, prints out its name and then the values.

In [ ]:
list(tables)

In [ ]:
for species, tab in tables.items():  # pull out each Key->Value pair in the tables variable
    print("table:", species)
    print(tab)
    print()

Remember that each table is now a dictionary entry, so in the `for` loop we specify **`for species, tab in tables.items():`**. 

`species` stands for key, 
`tab` for value. 

This is how you iterate over dictionaries. 

We then print the `species` value first, then the `tab` variable (the data within the table). 
The last `print` statement just creates a space between one table and the next.

### Creating the dataframe

Now is the time to put the pieces back together. We really want to create a single data frame and create a new variable specifying the species. Take a look at how we use a for loop below to iterate over `table.items()` again.

In [ ]:
dfs = [] # an empty list where we will store our separate dataframes

for species, tab in tables.items(): # iterate over the table dictionaries
    
#    single_frame = pd.DataFrame(tab) # create a dataframe from each table 
    
    single_frame = pd.DataFrame(tables[species]) # create a dataframe from each table 
    
    single_frame['species'] = species # create a new column called species and fill it with the name of the data frame
    
    single_frame = single_frame.reset_index(drop=True).iloc[1:] # reset index and remove first row (header row)
    
    single_frame.columns = ['date','plot','sex','weight','species'] # rename the columns
    
    dfs.append(single_frame) # add to the list of separate data frames

df = pd.concat(dfs).reset_index(drop = True) # join the dataframes together into one data frame
df # return the complete data frame    

There are a lot of steps in this for loop. For that reason, each line of code is commented to describe what each piece is doing. 

We are now in pretty good shape, but there are still some things we need to do before we can consider this tidy. One thing that should be noticeable right of the bat is there are rows of data that are almost completely `NaN` values, save for the `Species` column. Why is that? 

Let's take a look at the picture again...

![title](../images/messy2.png)

Notice that between each table there is a blank row. That is the very reason for these lines. Essentially, these are residual from our initial reading in of the file and they have remained throughout the process. Well, now it is time to get rid of them.

For this, we will use the Pandas `.dropna()` function to remove rows that have NaN values, then reset the row indexing as we did previously.

**Example** 
```
df.dropna().reset_index(drop = True)
```
However, if we just drop the `NaN` rows, we will lose some of the Species DM rows that have weight blank.
Instead, we need to limit the `NaN` check to the `date` column.

In [ ]:
# Re-assign the result of removing NaN date rows and re-index the rows of the data frame
df = df.dropna(subset = ['date']).reset_index(drop = True)
df

# Column Clean Up

We have achieved most of carpentry goal, now we just need to reform the species column.
Having "Species:" repeated for every single value is unnecessary. 
We are going to go ahead and remove this segment, leaving only the species abbreviations.

Again, there are several ways to do this, but the method below takes on a familiar format that we have seen before. 

Note: The code below relies on 
  1. String `.split()` to convert a string into a list of strings
  1. List `.pop()` to remove and return the last item in the list.

In [ ]:
abbrv = []  # Start an empty list

# for every row in the species column
for i in df.species:
    # Append to the list
    abbrv.append(
        i.split(" ").pop() # the result of splitting the Species: DX into two elements, taking the first element away.
    )
    
# The above for loop produced a list of Species, make that list the new species column
df['species'] = abbrv

df.head(20)

Let's take a look at the `df['weight']` column. 
This is a variable that we would like to find some stats about, but in its current state we can't do that. 

Take a look at the column again. 
Some of the values have a "g" added to the number indicating grams. 
It is this "g" that we want to remove so that we can start running some stats on it. 
Below is one way to do that.

In [ ]:
# for this we need to import the numpy package (numerical python)
import numpy as np

# Start an empty list
nums = []
# for each value in the column
for i in df.weight:
    # if the value i, pulled from the column is empty as a number
    if pd.isnull(i):
        val = np.nan # set the temporary variable to Not a Number
    else:
        # Other wise join all the digits into a single string in the temporary val variable
        val = ''.join(c for c in str(i) if c.isdigit())
    # Add the variable into the list, possible as NaN or a number.
    nums.append(val)
    
# Assing this column of clean variables for weight to the weight column
df['weight'] = nums

df.head(20)

We see that the "g" characters are removed.

The method is similar to how we modified the `species` column except we added a conditional. The reason we need the conditional is because `NaN` values will halt our for loop and we don't want to get rid of them, so we first need to check if the value is `NaN`. If the values are `NaN`, we will return them as `NaN`, otherwise, we use this line of code to take only those values that are numbers and return them.

#### But there is one problem... 
Our column is still a `String` type, and we cannot compute a numerical mean on a string. 

Therefore, we will convert it to a float using the Numerical Python library (NumPy).

In [ ]:
df['weight'] = df['weight'].astype(float)

In [ ]:
df.weight

Now what to do about those pesky `NaN`s? Well, we could remove rows with those values, or we can fill them in. Below is how you fill them in using the mean weight, this way we are using the average of the dataset.

But let's take a look at the column again. We will copy the data frame to a frame called `cleaned`.

In [ ]:
cleaned = df
cleaned['weight']

In [ ]:
cleaned['weight'] = cleaned['weight'].fillna(cleaned['weight'].mean())

And that is how you would fill in the `NaN` values with the mean weight. 
This allows us to keep the information of the other columns, which may be useful, 
but not affect the overall mean of the weight column. 
Take a look at our final frame below.

**To think about:** Is there a better way to fill in these `NaN`s?



In [ ]:
cleaned

## Data Carpentry is sometimes tedious to construct as a process for data cleaning

However, remember that the alternative was to clean 100's or 1000's of those files by hand (with Excel or text editor).

By developing a data transformation and cleaning script, we can ingest any number of these files for further analytical inspection.

# Save your notebook, then `File > Close and Halt`